### 8.3 让对象支持上下文管理协议

如何让对象支持上下文管理协议（with 语句）

为了让一个对象兼容with 语句，需要实现**`__enter__()`** 和 **`__exit__()`** 方法，例如，考虑下面的一个类，他能为我们创建一个网络连接

In [6]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None
        
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connect')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

socket是python提供的用于进行socket编程的接口，SOCK_STREAM表示使用TCP协议

这个类的特点是他表示了一个网络连接，但是初始化的时候并不会做任何事情。连接的建立和关闭都是使用with 语句自动完成的。

编写上下文管理器的主要原理是：**代码块会放在with语句块中执行**,当出现`with` 语句时，对象的***`__enter__()`*** 方法被触发，它返回的值（如果有的话）会被赋值给 `as` 声明的变量。
然后，`with`语句块里的代码开始执行，
最后，***`__exit__()`*** 方法被触发进行清理工作。
例如：

In [9]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))

with conn as s:
    # conn.__enter__() excutes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.baidu.com\r\n')
    s.send(b'\r\n')
    resp = b"".join(iter(partial(s.recv,8192), b''))
    print(resp)
    # conn.__exit__() executes: connection closed

b'HTTP/1.1 503 Backend unavailable, connection timeout\r\nServer: Varnish\r\nRetry-After: 0\r\nContent-Type: text/html; charset=utf-8\r\nAccept-Ranges: bytes\r\nContent-Length: 512\r\nAccept-Ranges: bytes\r\nDate: Fri, 22 Mar 2019 07:56:15 GMT\r\nVia: 1.1 varnish\r\nConnection: close\r\nX-Served-By: cache-tyo19947-TYO\r\nX-Cache: MISS\r\nX-Cache-Hits: 0\r\nX-Timer: S1553241375.805530,VS0,VE1009\r\n\r\n\n<?xml version="1.0" encoding="utf-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"\n "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n<html>\n  <head>\n    <title>503 Backend unavailable, connection timeout</title>\n  </head>\n  <body>\n    <h1>Error 503 Backend unavailable, connection timeout</h1>\n    <p>Backend unavailable, connection timeout</p>\n    <h3>Guru Mediation:</h3>\n    <p>Details: cache-tyo19934-TYO 1553241376 1990286731</p>\n    <hr>\n    <p>Varnish cache server</p>\n  </body>\n</html>\n'


不管`with`代码块中发生什么，上面的流都会执行完，就算`with`代码块发生异常也是这样。
事实上,***`__exit__()`***方法的第三个参数包含了异常类型、或者忽略他返回一个None值。
如果 `__exit__()`返回 True，那么异常会被清空，with语句后的内容正常执行

还有一个细节，就是 LazyConnection 类是否允许多个 with 语句来进行嵌套连接。 很显然，上面的定义中异常只能允许一个 socket 连接。如果正在使用一个 socket 的时候又重复使用 with 语句，就会产生异常，不过可以这样修改：

In [10]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.type = type
        self.family = family
        self.connection = []
        
    def __enter__(self):
        # family指定建立网络连接，type指定协议
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connection.append(sock)
        return sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.connection.pop().close()

        

在改版本中，LazyConnection类 可以看做是某个 ***连接工厂*** 
在内部，一个列表被用来构造一个栈。
每次 ***`__enter__()`*** 方法执行的时候，他赋值一个新的连接并将其加入到栈中, 
***`__exit__()`*** 只是简单的从栈中弹出最后一个连接，并关闭它。
其使用如下：

In [11]:
from functools import partial
conn = LazyConnection(('www.baidu.com',80))
with conn as s1:
    pass
    with conn as s2:
        pass

在需要管理一些资源，**比如文件、网络连接和锁的编程环境中，使用上下文管理器是非常普遍的** 。这些资源的一个特征是他们必须被手动关闭或释放来确保程序的正确运行。